# Setup

In [1]:
%matplotlib ipympl
import matplotlib, matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Number of custom components

In [2]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
        {
            "Unique custom components": lambda app: app.components.filter(lambda component: component.type == "Custom")
            .map(lambda component: component[".tagName"])
            .filter(lambda tagName: tagName != None)
            .unique()
            .length
        }
    ).order_by(lambda app: (app["Unique custom components"],), reverse=True).table()

Env,Org,App,Unique custom components
tt02,dibk,fa-v3,10
tt02,dibk,ig-v3,10
tt02,dibk,mb-v3,10
tt02,dibk,innsending-planforslag,6
tt02,dat,byggesak-samtykke-v3,6
tt02,digdir,dan-data-app,1


Count: 6


## Custom components per app

In [6]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
        {
            "Custom component tags": lambda app: app.components.filter(lambda component: component.type == "Custom")
            .map(lambda component: component[".tagName"])
            .unique()
            .sort()
        }
    ).order_by(lambda app: (app.org, app.app)).table()

Env,Org,App,Custom component tags
tt02,dat,byggesak-samtykke-v3,"[custom-field-adresse, custom-field-data, custom-field-kommunens-saksnummer, custom-field-part-navn, custom-field-prosjekt, custom-field-telefonnummer]"
tt02,dibk,fa-v3,"[custom-field-adresse, custom-field-boolean-text, custom-field-data, custom-field-kommunens-saksnummer, custom-field-prosjekt, custom-grouplist-utfall-svar-type, custom-header-text, custom-list-data, custom-subheader-text, custom-table-data]"
tt02,dibk,ig-v3,"[custom-field-adresse, custom-field-boolean-text, custom-field-data, custom-field-kommunens-saksnummer, custom-field-prosjekt, custom-grouplist-utfall-svar-type, custom-header-text, custom-list-data, custom-subheader-text, custom-table-data]"
tt02,dibk,innsending-planforslag,"[custom-field-adresse, custom-field-data, custom-field-kommunens-saksnummer, custom-header-text, custom-paragraph-text, custom-table-data]"
tt02,dibk,mb-v3,"[custom-field-adresse, custom-field-boolean-text, custom-field-data, custom-field-kommunens-saksnummer, custom-field-prosjekt, custom-grouplist-utfall-svar-type, custom-header-text, custom-list-data, custom-subheader-text, custom-table-data]"
tt02,digdir,dan-data-app,[my-component]


Count: 6


## Apps using Subform in prod

In [4]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod" and app.components.some(lambda component: component.type == "Subform")
    ).table()

Count: 0


## RuleConfiguration conditional rendering rules

In [30]:
with Apps.init() as apps:
    apps.where(lambda app: app.env == "prod").select(
        {
            "Frontend version": lambda app: app.frontend_version,
            "Rule config": lambda app: app.rule_configurations.flat_map(
                lambda rule_configuration: rule_configuration[".data.conditionalRendering.[].selectedFunction", :],
            ),
        }
    ).where(lambda app: app["Rule config"].length > 0).table()

Env,Org,App,Frontend version,Rule config
prod,digdir,be-om-api-nokkel,4,"[[isNotEmpty, isWebApp, isNotEmpty]]"
prod,dibk,uttalelse-hoffe,3,"[[sjekkUttalelseFristDato, sjekkUttalelseFristDato]]"
prod,dibk,uttalelse-varselplanoppstart,3,"[[fristForInnspillGyldig, fristForInnspillGyldig]]"
prod,brg,anonym-oppstartsveilederen,3,"[[isOverskudd, isUnderskudd, isUnderskudd, isBehov, isArbeidsledig, isOrgOppdrag, isHobby, isMesser, isArbeidsgiver, isArbeidsgiverVidere, isKjennerMarkedKundeOrTrorKjennerKundeOrKjennerIkkeKunde, iskonkurrenteneOversikt, isvissOversiktKonkurrentene, isikkeOversiktKonkurrentene, isMindreVarierendeJa, isMindreVarierendeNei, isMindreVarierendeRelevant, isSkattJa, isSkattNei, isBudsjettOk, isBudsjettNei, isJaMvaOrJaMvaHvordanOrNeiMva, isRegnskapJa, isRegnskapNei, isRegnskapRegnskapsfører, isRettigheterJa, isRettigheterNei, isAnsvarJa, isAnsvarNei, isdiskuterJa, isdiskuterLitt, isdiskuterNei, harVesentligeUtgifterOgEgenkapitalUnder15ProsentOgTrengerLaanEllerKreditt, harVesentligeUtgifterOgEgenkapitalMellom15og25ProsentOgTrengerLaanEllerKreditt, harVesentligeUtgifterOgEgenkapitalMellom25og40Prosent, harVesentligeUtgifterOgEgenkapitalMellom40og70ProsentOgTrengerLaan, harEgenkapitalMellom70og100ProsentAvUtgifterOgLaanMindreEnn5Prosent, harEgenkapitalMellom70og100ProsentAvUtgifterOgLaanMerEnn5Prosent, TrafficLightGreen, TrafficLightYellow, TrafficLightRed]]"
prod,brg,oppstartsveileder-as-anonym,3,"[[isOverskudd, isSpm01alt3, isSpm02alt1, isSpm3ealt1, isSpm3ealt2, isSpm3ealt3, isUnderskudd, isUnderskudd, isBehov, isArbeidsledig, isOrgOppdrag, isHobby, isMesser, isArbeidsgiver, isArbeidsgiverVidere0, isArbeidsgiverVidere, isArbeidsgiverVidere1, isArbeidsgiverVidere2, isArbeidsgiverVidere3, isKjennerMarkedKundeOrTrorKjennerKundeOrKjennerIkkeKunde, iskonkurrenteneOversikt, isvissOversiktKonkurrentene, isikkeOversiktKonkurrentene, isMindreVarierendeJa, isMindreVarierendeNei, isMindreVarierendeRelevant, isSkattJa, isSkattNei, isBudsjettOk, isBudsjettNei, isJaMvaOrJaMvaHvordanOrNeiMva, isRegnskapJa, isRegnskapNei, isRegnskapRegnskapsfører, isRettigheterJa, isRettigheterNei, isAnsvarJa, isAnsvarNei, isdiskuterJa, isdiskuterLitt, isdiskuterNei, harVesentligeUtgifterOgEgenkapitalUnder15ProsentOgTrengerLaanEllerKreditt, harVesentligeUtgifterOgEgenkapitalMellom15og25ProsentOgTrengerLaanEllerKreditt, harVesentligeUtgifterOgEgenkapitalMellom25og40Prosent, harVesentligeUtgifterOgEgenkapitalMellom40og70ProsentOgTrengerLaan, harEgenkapitalMellom70og100ProsentAvUtgifterOgLaanMindreEnn5Prosent, harEgenkapitalMellom70og100ProsentAvUtgifterOgLaanMerEnn5Prosent, TrafficLightGreen, TrafficLightYellow, TrafficLightRed]]"
prod,brg,oppstartsveilederen,3,"[[isOverskudd, isUnderskudd, isUnderskudd, isBehov, isArbeidsledig, isOrgOppdrag, isHobby, isMesser, isArbeidsgiver, isArbeidsgiverVidere, isKjennerMarkedKundeOrTrorKjennerKundeOrKjennerIkkeKunde, iskonkurrenteneOversikt, isvissOversiktKonkurrentene, isikkeOversiktKonkurrentene, isMindreVarierendeJa, isMindreVarierendeNei, isMindreVarierendeRelevant, isSkattJa, isSkattNei, isBudsjettOk, isBudsjettNei, isJaMvaOrJaMvaHvordanOrNeiMva, isRegnskapJa, isRegnskapNei, isRegnskapRegnskapsfører, isRettigheterJa, isRettigheterNei, isAnsvarJa, isAnsvarNei, isdiskuterJa, isdiskuterLitt, isdiskuterNei, harVesentligeUtgifterOgEgenkapitalUnder15ProsentOgTrengerLaanEllerKreditt, harVesentligeUtgifterOgEgenkapitalMellom15og25ProsentOgTrengerLaanEllerKreditt, harVesentligeUtgifterOgEgenkapitalMellom25og40Prosent, harVesentligeUtgifterOgEgenkapitalMellom40og70ProsentOgTrengerLaan, harEgenkapitalMellom70og100ProsentAvUtgifterOgLaanMindreEnn5Prosent, harEgenkapitalMellom70og100ProsentAvUtgifterOgLaanMerEnn5Prosent, TrafficLightGreen, TrafficLightYellow, TrafficLightRed]]"
prod,brg,rrh-innrapportering,4.8.2,"[[valueIsTrue, valueIsTrue, valueIsTrue, valueIsNotEmptyOrFalse, valueIsNotEmptyOrFalse, valueIsGrunnlagtypeIndi, valueIsGrunnlagtypeIndi, showKontMellomliggende, showKontMellomliggende, valueIsNotEmptyOrFal

Count: 112
